In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [69]:
#Change the date to the previous day
query = """
WITH Schedule as
(
SELECT
SPLIT(Session5Date, ' ')[0] Date, Year, RoundNumber, EventName, EventFormat
FROM `perceptive-ivy-290216.f1_api.schedules_update`
WHERE Session5='Race'
GROUP BY 1,2,3,4,5
ORDER BY 2,1
)
,

Results as (
SELECT DriverNumber, Abbreviation, TeamName,FullName, Position, GridPosition, Time, Status, Year, GP, Date, RoundNumber, EventFormat, SUM(Points) Points
FROM
(
(
SELECT
DriverNumber, Abbreviation, TeamName,FullName, CAST(Position as INT64) Position,CAST(GridPosition as INT64) GridPosition, Time, Status, CAST(Points as INT64) Points, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat
FROM `perceptive-ivy-290216.f1_api.results_race` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE A.Year=2016
ORDER BY FullName, RoundNumber
)
UNION ALL
(
SELECT
DriverNumber, Abbreviation, TeamName,FullName, CAST(Position as INT64) Position,CAST(GridPosition as INT64) GridPosition, Time, Status, CAST(Points as INT64) Points, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE A.Year=2016
ORDER BY FullName, RoundNumber
)
)
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13
)

SELECT
Date, DriverNumber, Abbreviation, TeamName,FullName, Position, GridPosition, Time, Status, Points,
SUM(Points) OVER (PARTITION BY Year, FullName ORDER BY ROUNDNUMBER) PTS,
Year, GP, RoundNumber, EventFormat
FROM
Results
ORDER BY FullName, RoundNumber

"""
project_id = "perceptive-ivy-290216"
df_bq = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [70]:
years=2022

In [71]:
df_bq_driver=df_bq[df_bq["Year"]==years]
df_bq_driver=df_bq_driver.sort_values(by=['Date'])
df_bq_driver["DriverNumber"]=df_bq_driver["DriverNumber"].astype(int)
df_bq_driver=df_bq_driver.fillna(0)
df_bq_driver.head()

,Date,DriverNumber,Abbreviation,TeamName,FullName,Position,GridPosition,Time,Status,Points,PTS,Year,GP,RoundNumber,EventFormat
855,2022-03-20,23,ALB,Williams,Alexander Albon,13,14,0 days 01:38:27.516000,Finished,0,0,2022,Bahrain Grand Prix,1,conventional
6424,2022-03-20,14,ALO,Alpine,Fernando Alonso,9,8,0 days 01:37:55.974000,Finished,2,2,2022,Bahrain Grand Prix,1,conventional
7080,2022-03-20,63,RUS,Mercedes,George Russell,4,9,0 days 01:37:44.795000,Finished,12,12,2022,Bahrain Grand Prix,1,conventional
3864,2022-03-20,3,RIC,McLaren,Daniel Ricciardo,14,18,0 days 01:38:28.559000,Finished,0,0,2022,Bahrain Grand Prix,1,conventional
8136,2022-03-20,24,ZHO,Alfa Romeo,Guanyu Zhou,10,15,0 days 01:37:56.648000,Finished,1,1,2022,Bahrain Grand Prix,1,conventional


In [72]:
fig=px.line(
        df_bq_driver,
        x="GP", 
        y="PTS", 
        color='Abbreviation', 
        # markers=True,
        hover_data=['DriverNumber','TeamName', 'Position', 'GridPosition', 'Status', 'Year', 'GP', 'RoundNumber'], 
        template="xgridoff",
        title="<b>Cumulative Driver Points for the {} F1 Season</b>".format(years),
        height=800, 
        width=1200,
        color_discrete_map={
                "ALB": "#64C4FF",
                "ALO": "#229971",
                "BEA":"#E80020",
                "BOT": "#52e252",
                "GAS": "#0093cc",
                "HAM": "#27F4D2",
                "HUL": "#B6BABD",
                "LEC": "#E80020",
                "MAG": "#B6BABD",
                "NOR": "#FF8000",
                "OCO": "#0093cc",
                "PER": "#3671C6",
                "PIA": "#FF8000",
                "RIC": "#6692FF",
                "RUS": "#27F4D2",
                "SAI": "#E80020",
                "SAR": "#37BEDD",
                "STR": "#229971",
                "TSU": "#6692FF",
                "VER": "#3671C6",
                "ZHO": "#52e252"
          },
        )
fig.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=10)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    # margin=dict(l=10, r=5, t=55, b=20),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),

)

In [73]:
fig.write_html("Trended Driver Points for the {} F1 Season.html".format(years))

In [115]:
#Team PTS

In [18]:
#Change the date to the previous day
query = """
WITH Schedule as
(
SELECT
SPLIT(Session5Date, ' ')[0] Date, Year, RoundNumber, EventName, EventFormat
FROM `perceptive-ivy-290216.f1_api.schedules_update`
WHERE Session5='Race'
GROUP BY 1,2,3,4,5
ORDER BY 2,1
)
,

Results as (
SELECT TeamName, Year, GP, Date, RoundNumber, EventFormat, SUM(Points) Points
FROM
(
(
SELECT
TeamName, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat, SUM(CAST(Points as INT64)) Points
FROM `perceptive-ivy-290216.f1_api.results_race` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
GROUP BY 1,2,3,4,5,6
)
UNION ALL
(
SELECT
TeamName, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat, SUM(CAST(Points as INT64)) Points
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
GROUP BY 1,2,3,4,5,6
)
)
GROUP BY 1,2,3,4,5,6
)

SELECT
TeamName, Year, GP, Date, 
RoundNumber, EventFormat, Points, SUM(Points) OVER (PARTITION BY Year, TeamName ORDER BY ROUNDNUMBER) PTS,
FROM
Results

"""
project_id = "perceptive-ivy-290216"
df_bq_teams = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [19]:
df_bq_teams[(df_bq_teams["TeamName"]=="Aston Martin")&(df_bq_teams["Year"]==2024)]

,TeamName,Year,GP,Date,RoundNumber,EventFormat,Points,PTS
12233,Aston Martin,2022,Bahrain Grand Prix,2022-03-20,1,conventional,0,0
12234,Aston Martin,2022,Saudi Arabian Grand Prix,2022-03-27,2,conventional,0,0
12235,Aston Martin,2022,Australian Grand Prix,2022-04-10,3,conventional,0,0
12236,Aston Martin,2022,Emilia Romagna Grand Prix,2022-04-24,4,sprint,5,5
12237,Aston Martin,2022,Miami Grand Prix,2022-05-08,5,conventional,1,6
12238,Aston Martin,2022,Spanish Grand Prix,2022-05-22,6,conventional,0,6
12239,Aston Martin,2022,Monaco Grand Prix,2022-05-29,7,conventional,1,7
12240,Aston Martin,2022,Azerbaijan Grand Prix,2022-06-12,8,conventional,8,15
12241,Aston Martin,2022,Canadian Grand Prix,2022-06-19,9,conventional,1,16
12242,Aston Martin,2022,British Grand Prix,2022-07-03,10,conventional,2,18


In [20]:
df_bq_team=df_bq_teams[df_bq_teams["Year"]==years]
df_team=df_bq_team.groupby(by=['Date', 'TeamName','Points', 'Year','GP','RoundNumber','EventFormat'])["PTS"].sum()
df_team=pd.DataFrame(df_team)
df_team.reset_index(inplace=True)
df_team=df_team.sort_values(by=['Date'])
df_team.head()

,Date,TeamName,Points,Year,GP,RoundNumber,EventFormat,PTS
0,2022-03-20,Alfa Romeo,9,2022,Bahrain Grand Prix,1,conventional,9
1,2022-03-20,AlphaTauri,4,2022,Bahrain Grand Prix,1,conventional,4
2,2022-03-20,Alpine,8,2022,Bahrain Grand Prix,1,conventional,8
3,2022-03-20,Aston Martin,0,2022,Bahrain Grand Prix,1,conventional,0
4,2022-03-20,Ferrari,44,2022,Bahrain Grand Prix,1,conventional,44


In [21]:
df_team[df_team['TeamName']=='Aston Martin']

,Date,TeamName,Points,Year,GP,RoundNumber,EventFormat,PTS
3,2022-03-20,Aston Martin,0,2022,Bahrain Grand Prix,1,conventional,0
13,2022-03-27,Aston Martin,0,2022,Saudi Arabian Grand Prix,2,conventional,0
23,2022-04-10,Aston Martin,0,2022,Australian Grand Prix,3,conventional,0
33,2022-04-24,Aston Martin,5,2022,Emilia Romagna Grand Prix,4,sprint,5
43,2022-05-08,Aston Martin,1,2022,Miami Grand Prix,5,conventional,6
53,2022-05-22,Aston Martin,0,2022,Spanish Grand Prix,6,conventional,6
63,2022-05-29,Aston Martin,1,2022,Monaco Grand Prix,7,conventional,7
73,2022-06-12,Aston Martin,8,2022,Azerbaijan Grand Prix,8,conventional,15
83,2022-06-19,Aston Martin,1,2022,Canadian Grand Prix,9,conventional,16
93,2022-07-03,Aston Martin,2,2022,British Grand Prix,10,conventional,18


In [22]:
fig_team=px.line(
        df_team, 
        x="GP", 
        y="PTS", 
        color='TeamName', 
        # markers=True,
        hover_data=['TeamName','Year', 'GP', 'RoundNumber','EventFormat'], 
        template="xgridoff",
        title="<b>Cumulative Team Points for the {} F1 Season</b>".format(years),
        height=800, 
        width=1200,
        color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
        )
fig_team.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=10)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    # margin=dict(l=10, r=5, t=55, b=20),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),

)

In [121]:
fig_team.write_html("Trended Team Points for the {} F1 Season.html".format(years))